In [1]:
# %% ===============================================================
# EJEMPLO FINAL SÚPER SIMPLE PARA VERIFICAR EL MODELO
# ===============================================================
import torch
import torch.nn as nn
import json
import numpy as np

# --- 1. DEFINICIÓN DE LA ARQUITECTURA DEL MODELO ---
# Usamos la definición correcta que coincide con tu modelo guardado.
class RiskNN(nn.Module):
    def __init__(self, num_features, cat_dims, emb_dims, hidden, dropout):
        super().__init__()
        self.emb = nn.ModuleList([
            nn.Embedding(dim, emb) for dim, emb in zip(cat_dims, emb_dims)
        ])
        in_dim = num_features + sum(emb_dims)
        layers = []
        for h in hidden:
            layers += [nn.Linear(in_dim, h),
                       nn.BatchNorm1d(h),
                       nn.GELU(),
                       nn.Dropout(dropout)]
            in_dim = h
        layers.append(nn.Linear(in_dim, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x_num, x_cat):
        emb = [m(x_cat[:, i]) for i, m in enumerate(self.emb)]
        x   = torch.cat(emb + [x_num], dim=1)
        return self.net(x).squeeze(1)


# --- 2. CARGAR LA CONFIGURACIÓN Y LOS PESOS DEL MODELO ---
DEVICE = torch.device("cpu")
model_weights_path = "best_model_final.pth"
model_metadata_path = "model_metadata.json"

try:
    # Paso CLAVE: Leer los parámetros exactos desde el archivo de metadatos
    with open(model_metadata_path, 'r') as f:
        metadata = json.load(f)

    # Instanciar el modelo usando los parámetros del archivo
    nn_model = RiskNN(
        num_features=metadata['num_features'],
        cat_dims=metadata['cat_dims'],
        emb_dims=metadata['emb_dims'],
        hidden=metadata['hidden_layers'],
        dropout=metadata['dropout'],
    ).to(DEVICE)

    # Cargar los pesos entrenados
    nn_model.load_state_dict(torch.load(model_weights_path, map_location=DEVICE))
    nn_model.eval() # Poner el modelo en modo de evaluación

    print(f"✔️ ¡Éxito! Modelo '{model_weights_path}' cargado correctamente.")

except Exception as e:
    print(f"❌ Ocurrió un error: {e}")
    print("Asegúrate de que los archivos 'best_model_final.pth' y 'model_metadata.json' están en la misma carpeta que el notebook.")


# --- 3. CREAR UN CLIENTE DE EJEMPLO Y HACER UNA PREDICCIÓN ---
# (Esta parte es solo para demostrar que el modelo funciona)

# a) Variables numéricas de ejemplo (8 valores normalizados)
sample_numerical = torch.tensor([[-0.5, 0.2, -0.1, 0.5, -0.3, 0.1, 0.8, -1.0]], dtype=torch.float32).to(DEVICE)

# b) Variables categóricas de ejemplo (6 valores de índice)
#    IMPORTANTE: Los índices deben ser menores que el tamaño de su embedding correspondiente.
#    Por ejemplo, el primer índice (para emp_length) debe ser < 6.
sample_categorical = torch.tensor([[1, 3, 1, 0, 1, 0]], dtype=torch.long).to(DEVICE)

# c) Realizar la predicción
with torch.no_grad():
    logits = nn_model(sample_numerical, sample_categorical)
    probability = torch.sigmoid(logits).item()


# --- 4. MOSTRAR EL RESULTADO ---
print("\n--- Resultado de la Predicción de Ejemplo ---")
print(f"Probabilidad de incumplimiento: {probability*100:.2f}%")

✔️ ¡Éxito! Modelo 'best_model_final.pth' cargado correctamente.

--- Resultado de la Predicción de Ejemplo ---
Probabilidad de incumplimiento: 39.42%


C:\Users\BRAYAN\AppData\Local\Temp\ipykernel_18180\3504849935.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nn_model.load_state_dict(torch.load(model_weights_path, ma